In [1]:
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
%matplotlib inline
import numpy as np
from astropy_healpix import HEALPix as hp
from scipy.interpolate import griddata

def nside_to_pixel_area(nside):
    nside = np.asanyarray(nside, dtype=np.int64)
    npix = 12 * nside * nside
    pixel_area = 4 * np.pi / npix * u.sr
    return pixel_area
    
def nside_to_pixel_resolution(nside):
    nside = np.asanyarray(nside, dtype=np.int64)
    return (nside_to_pixel_area(nside) ** 0.5).to(u.arcmin)

data = np.load('I_355_tgextmap.npz')
HPId = data['HPId']
A0 = data['A0']
level = 9

ra = 260.10740722*u.deg
dec = -19.3523078*u.deg
coord = SkyCoord(ra, dec, frame='icrs')

hp = hp(nside=2**level, order='nested', frame='icrs')
search=hp.cone_search_skycoord(coord, radius=5 * u.deg)
ext = A0[np.isin(HPId, search)]
common = HPId[np.isin(HPId, search)]
cd = hp.healpix_to_skycoord(common)

ra = np.unique(cd.ra.value)
dec = np.unique(cd.dec.value)
X, Y = np.meshgrid(ra, dec)
beta = griddata(np.transpose(np.array([cd.ra.value, cd.dec.value])), np.array(ext), (X, Y), method='nearest')
im = plt.imshow(beta, cmap=plt.cm.viridis, aspect='auto', extent=[X.min(), X.max(), Y.min(), Y.max()],
                origin='lower')
plt.colorbar(im)
plt.show()

f=nside_to_pixel_resolution(2**level)
print(f)

ModuleNotFoundError: No module named 'astropy_healpix'